In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score

In [5]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
time = pd.read_csv('time.csv')

In [6]:
X['time'] = time

In [7]:
X

,amateur+doggystyle,suck+ass,vegas,2+girls+share+cock,leia+lovelyn,ngentot,solo+joi,vagina+mojada,truutv,bjob,...,redhead+rough,rubbing+panties,2+girls+cum+mouth,italiane,forest,lights+off,ocean,antury,squirt+in+face,time
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,582
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,619
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,610
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8063
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19190,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,510
19191,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,761
19192,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1071
19193,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1079


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
y_train.value_counts(normalize=True)

kolumna_celi
False           0.848333
True            0.151667
Name: proportion, dtype: float64

In [10]:
chi2_stats, p_values = chi2(X_train, y_train)
chi2_stats

array([4.79756933e-04, 1.78782529e-01, 1.78782529e-01, ...,
       1.78782529e-01, 1.78782529e-01, 1.05767742e+03])

In [11]:
good_colums = []
for x, y in zip(chi2_stats,X_train.columns.tolist()):
  if x > 0.2:
    good_colums.append(y)
    
    
X_train_X = X_train[good_colums]
X_test_X = X_test[good_colums]

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [13]:
# Algorytm PCA nie działa na kolumny binarne, a MCA jedynie zabiera interpretowalność danych

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler()),               
    ])

In [14]:
X_train_pipe = pipeline.fit_transform(X_train_X)
X_test_pipe = pipeline.fit_transform(X_test_X)

In [15]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [16]:
classifiers = {
    'CatBoost': CatBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
}

param_grids = {
    'DecisionTree': {
        'max_depth': [200, 300, 400],
        'min_samples_split': [5]
    },
    'XGBoost': {
        'n_estimators': [1150, 200, 300],
        'max_depth': [9, 12],
        'learning_rate': [2, 2.5],
        'subsample': [0.6]
    },
    'CatBoost': {
        'iterations': [300],
        'depth': [6, 8],
        'learning_rate': [2, 2.5],
    }
}

In [17]:
best_estimators = {}
for clf_name in classifiers:
    print(f"Grid Search dla {clf_name}")
    grid_search = GridSearchCV(classifiers[clf_name], param_grids[clf_name], cv=5, scoring='recall', n_jobs=-1)
    grid_search.fit(X_train_pipe, y_train)
    best_estimators[clf_name] = grid_search.best_estimator_
    print(f"Najlepsze parametry dla {clf_name}: {grid_search.best_params_}")
    print(f"Najlepszy wynik dla {clf_name}: {grid_search.best_score_}\n")

Grid Search dla CatBoost


c:\Users\gracj\miniconda3\envs\czech\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\gracj\miniconda3\envs\czech\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\gracj\miniconda3\envs\czech\lib\site-packages\catboost\core.py", line 5201, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\gracj\miniconda3\envs\czech\lib

0:	learn: 0.5531629	total: 164ms	remaining: 49s
1:	learn: 0.6917346	total: 189ms	remaining: 28.2s
2:	learn: 0.5518757	total: 215ms	remaining: 21.2s
3:	learn: 0.6883643	total: 239ms	remaining: 17.7s
4:	learn: 0.5511413	total: 262ms	remaining: 15.5s
5:	learn: 0.6865722	total: 295ms	remaining: 14.4s
6:	learn: 0.5498774	total: 322ms	remaining: 13.5s
7:	learn: 0.6760052	total: 348ms	remaining: 12.7s
8:	learn: 0.5424806	total: 407ms	remaining: 13.2s
9:	learn: 0.6614965	total: 433ms	remaining: 12.6s
10:	learn: 0.5370993	total: 457ms	remaining: 12s
11:	learn: 0.6486322	total: 480ms	remaining: 11.5s
12:	learn: 0.5301244	total: 504ms	remaining: 11.1s
13:	learn: 0.6332616	total: 528ms	remaining: 10.8s
14:	learn: 0.5249725	total: 552ms	remaining: 10.5s
15:	learn: 0.6233647	total: 577ms	remaining: 10.2s
16:	learn: 0.5198992	total: 601ms	remaining: 10s
17:	learn: 0.6116915	total: 625ms	remaining: 9.79s
18:	learn: 0.5141392	total: 648ms	remaining: 9.59s
19:	learn: 0.6011433	total: 672ms	remaining: 9.

In [18]:
DT = DecisionTreeClassifier(max_depth= 200, min_samples_split = 5)
XGBC = XGBClassifier(learning_rate = 2, max_depth = 9, n_estimators = 150, subsample=0.6)
CBC = CatBoostClassifier(depth=8, iterations=300, learning_rate=2)

In [19]:
DT.fit(X_train_pipe, y_train)
XGBC.fit(X_train_pipe, y_train)
CBC.fit(X_train_pipe, y_train)

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 0.5530114	total: 58ms	remaining: 17.3s
1:	learn: 0.6910769	total: 94.7ms	remaining: 14.1s
2:	learn: 0.5517031	total: 130ms	remaining: 12.8s
3:	learn: 0.6882917	total: 165ms	remaining: 12.2s
4:	learn: 0.5508004	total: 200ms	remaining: 11.8s
5:	learn: 0.6864182	total: 235ms	remaining: 11.5s
6:	learn: 0.5475267	total: 272ms	remaining: 11.4s
7:	learn: 0.6753214	total: 310ms	remaining: 11.3s
8:	learn: 0.5348932	total: 349ms	remaining: 11.3s
9:	learn: 0.6454660	total: 388ms	remaining: 11.2s
10:	learn: 0.5223465	total: 430ms	remaining: 11.3s
11:	learn: 0.6160093	total: 470ms	remaining: 11.3s
12:	learn: 0.5144943	total: 512ms	remaining: 11.3s
13:	learn: 0.6018484	total: 551ms	remaining: 11.3s
14:	learn: 0.4971520	total: 593ms	remaining: 11.3s
15:	learn: 0.5654418	total: 633ms	remaining: 11.2s
16:	learn: 0.4849976	total: 674ms	remaining: 11.2s
17:	learn: 0.5368061	total: 714ms	remaining: 11.2s
18:	learn: 0.4673213	total: 756ms	remaining: 11.2s
19:	learn: 0.5055892	total: 796ms	remaini

In [20]:
y_pred_DT = DT.predict(X_test_pipe)

print(recall_score(y_test, y_pred_DT))
print(precision_score(y_test, y_pred_DT))
print(confusion_matrix(y_test, y_pred_DT))

0.0782312925170068
0.15181518151815182
[[2994  257]
 [ 542   46]]


In [21]:
y_pred_XGBC = XGBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_XGBC))
print(precision_score(y_test, y_pred_XGBC))
print(confusion_matrix(y_test, y_pred_XGBC))

0.10034013605442177
0.17933130699088146
[[2981  270]
 [ 529   59]]


In [22]:
y_pred_CBC = CBC.predict(X_test_pipe)

print(recall_score(y_test, y_pred_CBC))
print(precision_score(y_test, y_pred_CBC))
print(confusion_matrix(y_test, y_pred_CBC))

0.12755102040816327
0.16375545851528384
[[2868  383]
 [ 513   75]]
